# Preprocessing data

### Standardization, or mean removal and variance scaling


Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; 

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.


Generalmente gli algoritmi di ML lavorano meglio con caratteristiche che si trovano in una scala simile e sono simili alla distribuzione normale


In [1]:
from sklearn import preprocessing
import numpy as np
import pandas as pd

X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

X_scaled = preprocessing.scale(X_train) # non viene utilizzato questo metodo

X_scaled     

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [2]:
X_train.std(axis=0) # deviazione std

array([0.81649658, 0.81649658, 1.24721913])

In [3]:
X_train.mean(axis=0)

array([1.        , 0.        , 0.33333333])

In [4]:
X_scaled.mean(axis=0)

array([0., 0., 0.])

In [5]:
X_scaled.std(axis=0)

array([1., 1., 1.])

The standard score of a sample x is calculated as:
<tt>z = (x - u) / s</tt>
where u is the mean of the training samples or zero if <tt>with_mean=False</tt>, and s is the standard deviation of the training samples or one if <tt>with_std=False</tt>.


The preprocessing module further provides a utility class `StandardScaler` that implements the Transformer API to compute the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.

Quindi permette di calcolare la media e la deviazione std in un dataset di allenamento per poi successivamente riapplicare la stessa trasformazione sul dataset di test

In [6]:
sss = preprocessing.StandardScaler() # si istanzia il modello che si vuole usare
scaler = sss.fit(X_train) # calcola la media e la deviazione std per utilizzarli per il successivo ridimensionamento

In [7]:
scaler.mean_ 

array([1.        , 0.        , 0.33333333])

In [8]:
scaler.scale_ 

array([0.81649658, 0.81649658, 1.24721913])

In [9]:
newX_Train = scaler.transform(X_train) 

In [10]:
scaler.fit(X_train)
scaler.transform(X_train) # per l'andamento del modello stesso

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [11]:
scaler.fit_transform(X_train) # abbiamo il dataset nella sua interezza

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [12]:
scaler.fit_transform(X_train).mean()

4.9343245538895844e-17

In [13]:
scaler.fit_transform(X_train).std()

1.0

The scaler instance can then be used on new data to transform it the same way it did on the training set



In [14]:
X_test = [[-1., 1., 0.]]
scaler.transform(X_test)    

array([[-2.44948974,  1.22474487, -0.26726124]])

### Scaling features to a range

An alternative standardization is scaling features to lie between a given minimum and maximum value, often between zero and one, or so that the maximum absolute value of each feature is scaled to unit size.

In [15]:
# il KNN deve prima essere normalizzato:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

In [16]:
min_max_scaler = preprocessing.MinMaxScaler((0,1)) # default [0,1]

In [17]:
min_max_scaler.fit(X_train)

MinMaxScaler()

In [18]:
min_max_scaler.transform(X_train)

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [19]:
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [20]:
X_train_minmax

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [21]:
X_test = np.array([[-3., -1.,  4.]])
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax # perché i parametri calcolati si abasa sulla matrice precedente

array([[-1.5       ,  0.        ,  1.66666667]])

In [22]:
X_train.mean()

0.4444444444444444

In [23]:
X_train_minmax.mean()

0.48148148148148145

In [24]:
X_train.std()

1.0657403385139377

In [25]:
X_train_minmax.std()

0.4115946439054235

The transformation is computed as:

<tt>X_scaled = scale * X + min - X.min(axis=0) * scale</tt>

where <tt>scale = (max - min) / (X.max(axis=0) - X.min(axis=0))</tt>

In [26]:
min_max_scaler.scale_  

array([0.5       , 0.5       , 0.33333333])

If your data contains many outliers, scaling using the mean and variance of the data is likely to not work very well. In these cases, you can use robust_scale and RobustScaler (x_scaled = x / max(abs(x)))
as drop-in replacements instead. They use more robust estimates for the center and range of your data.



In [27]:
from sklearn.preprocessing import RobustScaler #  attutisce l'importanza degli outliers = valore anomali
X = [[ 1., -2.,  2.],
     [ -2.,  1.,  3.],
     [ 4.,  1., -2.]]

In [28]:
transformer = RobustScaler().fit(X)

In [29]:
transformer.scale_

array([3. , 1.5, 2.5])

In [30]:
transformer.transform(X)

array([[ 0. , -2. ,  0. ],
       [-1. ,  0. ,  0.4],
       [ 1. ,  0. , -1.6]])

## Normalization


Normalization is the process of scaling individual samples to have unit norm(norma unitaria). This process can be useful if you plan to use a quadratic form such as the dot-product(prodotto scalare) or any other kernel to quantify the similarity of any pair of samples.

‘l1’
The l1 norm uses the sum of all the values as and thus gives equal penalty to all parameters, enforcing sparsity.
x_normalized = x / sum(X)


‘l2’
The l2 norm uses the square root of the sum of all the squared values. This creates smoothness and rotational invariance. Some models, like PCA( [Analisi delle componenti principali](https://it.wikipedia.org/wiki/Analisi_delle_componenti_principali ) , assume rotational invariance, and so l2 will perform better.
x_normalized = x / sqrt(sum((i**2) for i in X))

In [31]:
# Normalization -> lavora sulle righe, lavorando in orrizzontale
X = np.array([[1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]])
X_normalized = preprocessing.normalize(X, norm='l2')

In [32]:
# the same as previous X_normalized = preprocessing.Normalizer(norm='l2').fit_transform(X)

In [33]:
X_normalized                                      

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [34]:
np.sqrt(0.40824829**2 + 0.40824829**2 + 0.81649658**2)

0.9999999988637723

You can refer to [https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html) for more details.

##  Encoding categorical features
Le features categoriche devono essere di trasformarli in vettori numerici

In [35]:
X = [['male', 'from US', 'uses Safari'], 
     ['female', 'from Europe', 'uses Firefox']]
X

[['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]

In [36]:
enc = preprocessing.OneHotEncoder()
enc.fit(X)  

OneHotEncoder()

In [37]:
enc.transform([['female', 'from US', 'uses Safari'],
               ['male', 'from Europe', 'uses Firefox']]).toarray() # se non applico questo viene stampato con una matrice sparsa

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 1., 0.]])

In [38]:
enc.get_feature_names_out() # per avere le etichette

array(['x0_female', 'x0_male', 'x1_from Europe', 'x1_from US',
       'x2_uses Firefox', 'x2_uses Safari'], dtype=object)

In [39]:
enc = preprocessing.OneHotEncoder(sparse=False) # tolgo lo sparse
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X) 

OneHotEncoder(sparse=False)

In [40]:
enc.transform([['female', 'from US', 'uses Safari'],
               ['male', 'from Europe', 'uses Firefox']]) # non é necessario il toarray()

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 1., 0.]])

In [41]:
enc.transform([['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']])

array([[0., 1., 0., 1., 0., 1.],
       [1., 0., 1., 0., 1., 0.]])

In [42]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

In [43]:
enc = preprocessing.OneHotEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X) 

OneHotEncoder()

In [44]:
pd.DataFrame(X, columns=['gender', 'locations', 'browser'])

,gender,locations,browser
0,male,from US,uses Safari
1,female,from Europe,uses Firefox


In [45]:
# per creare un dataset che sia leggibile
X = [['male', 'from Asia', 'uses Safari'], ['female', 'from Europe', 'uses Firefox'], ['male', 'from Asia', 'uses Safari']]
enc.fit(X)

new_features = []
for i in enc.categories_:
  for j in i:
    new_features.append(j)


Xnew = enc.transform(X).toarray()
pd.DataFrame(Xnew, columns=new_features)

,female,male,from Asia,from Europe,uses Firefox,uses Safari
0,0.0,1.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,1.0


In [46]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Asia', 'from Europe'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

with fixed categories value

In [47]:
genders = ['female', 'male']
locations = ['from Africa', 'from Asia', 'from Europe', 'from US']
browsers = ['uses Chrome', 'uses Firefox', 'uses IE', 'uses Safari']
enc = preprocessing.OneHotEncoder(categories=[genders, locations, browsers])

# Note that for there are missing categorical values for the 2nd and 3rd
# feature


X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X) 

OneHotEncoder(categories=[['female', 'male'],
                          ['from Africa', 'from Asia', 'from Europe',
                           'from US'],
                          ['uses Chrome', 'uses Firefox', 'uses IE',
                           'uses Safari']])

In [48]:
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()


array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

with unknown values

In [49]:
enc.transform([['female', 'from Reggio', 'uses Chrome']]).toarray()

ValueError: Found unknown categories ['from Reggio'] in column 1 during transform

In [ ]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore') # se lo incontra , mette uno 0
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X) 

In [ ]:
enc.transform([['female', 'from Reggio', 'uses Firefox']]).toarray()

In [ ]:
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist') # in version 1.1 é stato aggiunto automaticamente
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

In [ ]:
enc.categories_

In [ ]:
enc.transform([['female', 'from Reggio', 'uses Firefox']]).toarray()

In [ ]:
enc.inverse_transform([[1., 0., 0., 0., 1., 0.]]) # inverte convertendo i dati nella rappresentazione orginale

In [ ]:
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=1, max_categories=2)
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox'], ['female', 'from Europe', 'uses Firefox'], 
     ['female', 'from Europe', 'uses Firefox'],['female', 'from Europe', 'uses Firefox'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

In [ ]:
enc.get_feature_names_out()

In [ ]:
enc.transform([['female', 'from Reggio', 'uses Firefox']]).toarray()

In [ ]:
enc.transform([['male', 'from Europe', 'uses Firefox']]).toarray()

In [ ]:
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

In [ ]:
enc.inverse_transform([[1., 0., 0., 1., 0., 1.]])

drop first

In [ ]:
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox'], ['male', 'from Asia', 'uses Safari'], ['male', 'from Indo', 'uses Safari']]
drop_enc = preprocessing.OneHotEncoder(drop='first').fit(X)
# utile per situazioni in ui le features perfettamente collineari causano problemi, come quando si alimentano i dati risultati in una NN o in una regressione non regolarizzata

In [ ]:
drop_enc.categories_

In [ ]:
drop_enc.get_feature_names_out()

In [ ]:
drop_enc.transform(X).toarray()

In [ ]:
drop_enc.transform([['female','from Asia', 'uses Firefox']]).toarray()

In [ ]:
drop_enc.inverse_transform([[0., 0., 0., 0., 0.]])

In [ ]:
enc = preprocessing.OrdinalEncoder() # fa il map

In [ ]:
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

In [ ]:
enc.categories_

In [ ]:
enc.transform(X)

In [ ]:
enc.transform([['female','from Reggio', 'uses Safari']])

## Discretization

Discretization (otherwise known as quantization or binning) provides a way to partition continuous features into discrete values. Certain datasets with continuous features may benefit from discretization, because discretization can transform the dataset of continuous attributes to one with only nominal attributes.

In [50]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 3], encode='ordinal').fit(X) # numero di bin che voglio per la colonna(feature) 1 , colonna 2 e colonna 3
est

KBinsDiscretizer(encode='ordinal', n_bins=[3, 2, 3])

In [51]:
est.transform(X)

array([[0., 1., 2.],
       [1., 1., 1.],
       [2., 0., 0.]])

In [ ]:
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 3], encode='onehot').fit(X)
est

In [ ]:
est.transform(X).toarray()

In [ ]:
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 3], encode='onehot-dense').fit(X)
est

In [ ]:
est.transform(X)

There are different strategies implemented in KBinsDiscretizer:

- ‘uniform’: The discretization is uniform in each feature, which means that the bin widths are constant in each dimension.

- quantile’: The discretization is done on the quantiled values, which means that each bin has approximately the same number of samples.

- ‘kmeans’: The discretization is based on the centroids of a KMeans clustering procedure.

[Demonstrating the different strategies of KBinsDiscretizer](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html?highlight=kbinsdiscretizer)

Feature binarization is the process of thresholding numerical features to get boolean values. This can be useful for downstream probabilistic estimators that make assumption that the input data is distributed according to a multi-variate Bernoulli distribution.

In [51]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

binarizer = preprocessing.Binarizer().fit(X)  # fit does nothing
binarizer

Binarizer()

In [52]:
binarizer.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [53]:
binarizer = preprocessing.Binarizer(threshold=1.1)

In [54]:
binarizer.transform(X)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 0.]])

In [55]:
X[0][0] = 1.2

In [56]:
binarizer.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 0., 0.]])

## Custom transformers

Often, you will want to convert an existing Python function into a transformer to assist in data cleaning or processing. 

In [69]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p, validate=True)
X = np.array([[0, 1], [2, 3]])
transformer.transform(X)

array([[0.        , 0.69314718],
       [1.09861229, 1.38629436]])

The *validate* parameter indicates that the input X array should be checked before calling func. The possibilities are:

- If False, there is no input validation.

- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised.

In [65]:
type(X)

numpy.ndarray

In [68]:
type(transformer.transform(X)) 

numpy.ndarray

In [70]:
df = pd.DataFrame(X)

In [71]:
df

,0,1
0,0,1
1,2,3


In [72]:
transformer.transform(df)

array([[0.        , 0.69314718],
       [1.09861229, 1.38629436]])

In [73]:
transformer = FunctionTransformer(np.log1p, validate=False)

In [74]:
transformer.transform(df)

,0,1
0,0.000000,0.693147
1,1.098612,1.386294


Pay Attention!
The result of a transformer is typically a np.array!

In [75]:
preprocessing.StandardScaler().fit_transform(df)

array([[-1., -1.],
       [ 1.,  1.]])

In [76]:
preprocessing.StandardScaler().fit_transform(X)

array([[-1., -1.],
       [ 1.,  1.]])

In [77]:
X.shape

(2, 2)